In [4]:
#GANS for image generation


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




#Generator
class Generator(nn.Module):
    def __init__(self, latent_dim, img_size):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.img_size = img_size
        self.fc1 = nn.Linear(self.latent_dim, 128*4*4)
        self.bn1 = nn.BatchNorm2d(128)
        self.conv1 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv2 = nn.ConvTranspose2d(64, 32, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv3 = nn.ConvTranspose2d(32, 3, 4, 2, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = x.view(-1, 128, 4, 4)
        x = self.relu(self.bn1(x))
        x = self.relu(self.conv1(x))
        x = self.relu(self.bn2(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.bn3(x))
        x = self.conv3(x)
        x = torch.tanh(x)
        return x


#Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_size):
        super(Discriminator, self).__init__()
        self.img_size = img_size
        self.conv1 = nn.Conv2d(3, 32, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 1, 4, 1, 0)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.conv5(x)
        return x


#DataLoader
def get_data_loader(batch_size, img_size):
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    data_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(root='data/faces', transform=transform),
        batch_size=batch_size, shuffle=True)
    return data_loader


#Training
def train(batch_size, latent_dim, img_size, num_epochs):
    data_loader = get_data_loader(batch_size, img_size)
    G = Generator(latent_dim, img_size).to(device)
    D = Discriminator(img_size).to(device)
    G_optim = optim.Adam(G.parameters(), lr=0.0002)
    D_optim = optim.Adam(D.parameters(), lr=0.0002)
    criterion = nn.BCELoss()
    for epoch in range(num_epochs):
        for i, (imgs, _) in enumerate(data_loader):
            imgs = imgs.to(device)
            batch_size = imgs.size(0)
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            #Train D
            D.zero_grad()
            real_output = D(imgs)
            D_real_loss = criterion(real_output, real_labels)
            D_real_loss.backward()
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_imgs = G(z)
            fake_output = D(fake_imgs)
            D_fake_loss = criterion(fake_output, fake_labels)
            D_fake_loss.backward()
            D_loss = D_real_loss + D_fake_loss
            D_optim.step()
            #Train G
            G.zero_grad()
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_imgs = G(z)
            fake_output = D(fake_imgs)
            G_loss = criterion(fake_output, real_labels)
            G_loss.backward()
            G_optim.step()
            if i % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], D_loss: {:.4f}, G_loss: {:.4f}'
                      .format(epoch, num_epochs, i, len(data_loader), D_loss.item(), G_loss.item()))


#Testing
def test(batch_size, latent_dim, img_size):
    G = Generator(latent_dim, img_size).to(device)
    z = torch.randn(batch_size, latent_dim).to(device)
    fake_imgs = G(z)
    save_image(fake_imgs, 'fake_imgs.png')


    



